In [1]:

import org.apache.spark.sql.types._

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://ed10c28200f9:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591191339096)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._


In [2]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS-clean.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 30 more fields]


In [3]:
weather_org_df.registerTempTable("Aus_Rain_Data")

2020-06-03 13:36:19,180 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [4]:
/* val df0 = spark.sql("select * from Aus_Rain_Data") */

In [5]:
val df0 = spark.sql("select Location, date_Sc, MinTemp from Aus_Rain_Data")

df0: org.apache.spark.sql.DataFrame = [Location: string, date_Sc: timestamp ... 1 more field]


In [6]:
df0.printSchema()

root
 |-- Location: string (nullable = true)
 |-- date_Sc: timestamp (nullable = true)
 |-- MinTemp: string (nullable = true)



In [7]:
val df1 = df0.withColumn("date", df0("date_Sc").cast(DateType))
    .drop("date_Sc")
    .withColumnRenamed("date_Sc", "date")

df1: org.apache.spark.sql.DataFrame = [Location: string, MinTemp: string ... 1 more field]


In [8]:
val df2 = df1.withColumn("MinTemp_Sc", df1("MinTemp").cast(DoubleType))
    .drop("MinTemp")
    .withColumnRenamed("MinTemp_Sc", "MinTemp")

df2: org.apache.spark.sql.DataFrame = [Location: string, date: date ... 1 more field]


In [9]:
df2.printSchema()
df2.show(10)

root
 |-- Location: string (nullable = true)
 |-- date: date (nullable = true)
 |-- MinTemp: double (nullable = true)

+--------+----------+-------+
|Location|      date|MinTemp|
+--------+----------+-------+
|  Albury|2008-12-01|   13.4|
|  Albury|2008-12-02|    7.4|
|  Albury|2008-12-03|   12.9|
|  Albury|2008-12-04|    9.2|
|  Albury|2008-12-05|   17.5|
|  Albury|2008-12-06|   14.6|
|  Albury|2008-12-07|   14.3|
|  Albury|2008-12-08|    7.7|
|  Albury|2008-12-09|    9.7|
|  Albury|2008-12-10|   13.1|
+--------+----------+-------+
only showing top 10 rows



In [10]:
val window = Window.partitionBy("Location").orderBy("date").rowsBetween(-1, 1)

window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1cf0b336


In [11]:
val colNames = List("MinTemp")

colNames: List[String] = List(MinTemp)


In [14]:
/*for(column <- colNames){
  val df3 = df2.withColumn(column+"_Avg", mean(df2(column)).over(window))
} */

 val df3 = df2.withColumn("MinTemp_Avg", mean(df2("MinTemp")).over(window))

df3: org.apache.spark.sql.DataFrame = [Location: string, date: date ... 2 more fields]


In [16]:
df3.show(10)

+--------+----------+-------+------------------+
|Location|      date|MinTemp|       MinTemp_Avg|
+--------+----------+-------+------------------+
|  Cairns|2008-12-01|   25.2|              24.7|
|  Cairns|2008-12-02|   24.2|24.433333333333334|
|  Cairns|2008-12-03|   23.9|23.433333333333334|
|  Cairns|2008-12-04|   22.2|23.099999999999998|
|  Cairns|2008-12-05|   23.2|23.366666666666664|
|  Cairns|2008-12-06|   24.7|23.933333333333334|
|  Cairns|2008-12-07|   23.9|25.066666666666663|
|  Cairns|2008-12-08|   26.6| 25.53333333333333|
|  Cairns|2008-12-09|   26.1|26.233333333333334|
|  Cairns|2008-12-10|   26.0|              25.8|
+--------+----------+-------+------------------+
only showing top 10 rows



In [17]:
/* if (a == b) {
    doSomething()
} else {
    doSomethingElse()
} */


In [18]:
df3.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("testing")

In [19]:
!ls /home/testing

_SUCCESS  part-00000-ea9b548f-bf69-4c90-bc02-3af6a6ebd8f1-c000.csv



In [20]:
!mv /home/testing/*.csv /home/testing.csv

In [21]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/testing.csv /home/yury/Downloads/FIT5202-Project/big_data_tutorials/testing.csv"